<center><h1>Support Vector Machine (Машина Опорных Векторов)</h1></center>

Домашнее задание
Построение SVM и выбор оптимального ядра.
Цель: Цель данного ДЗ - "потрогать" алгоритм SVM на практике. Возьмите один из двух датасетов (или выберете свой) и попробуйте получить на нем максимальные метрики качества, используя алгоритм SVM. Проведите сравнение с Логистической регрессией. Используйте техники Feature Preprocessing, изученные ранее. Два датасета: 

__1) https://archive.ics.uci.edu/ml/datasets/IDA2016Challenge__

2) https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/overview 

С обоими датасетами вы уже знакомы - один был в ДЗб другой на лекции про Feature Preprocessing.
1) Возьмите датасет, выполните предобработку
2) Попробуйте изученные алгоритмы: kNN, Logistic Regression, SVM
3) Выполните настройку гиперпараметров для SVM (выбрать те параметры (ядро, регуляризацияб и тд) при которых метрики максимальны)
4*) Настройку гиперпараметров можно проводить используя GridSearchCV в реализации sklearn.
Критерии оценки: - Выполнена предобработка данных, преобразования признаков, посчитаны базовые статистики
- Построено хотя бы два алгоритма и показана разница результатов (или отсутствие разницы)
- Обучен SVM, выполнен подбор гиперпараметров
Рекомендуем сдать до: 13.01.2020 

Импорт необходимых библиотек перед переходом к ноутбуку:

In [34]:
#Итоговая таблица
df_quality

,Accuracy,Precision,Recall,f1_score,ROC-AUC
log_reg,0.94382,0.913043,0.724138,0.807692,0.855358
knn,0.926966,0.434783,1,0.606061,0.96131
svm_1,0.938202,0.913043,0.7,0.792453,0.843243
svm_best,0.949438,0.73913,0.85,0.790698,0.906013


In [1]:
import pandas as pd
import numpy as np
#from tqdm import tqdm_notebook

from sklearn.datasets import make_classification, make_circles
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error, \
    accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
plt.style.use("ggplot")

import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("aps_failure_training_set.csv", na_values="na")

In [3]:
df.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,76698,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,neg,33058,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,neg,41040,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,neg,12,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,neg,60874,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [4]:
df.shape

(60000, 171)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 171 entries, class to eg_000
dtypes: float64(169), int64(1), object(1)
memory usage: 78.3+ MB


In [6]:
df.rename(columns={"class":"target"},inplace=True)

In [7]:
df.replace("neg", -1,inplace=True)
df.replace("pos",  1,inplace=True)

In [8]:
print(sum(df.isnull().sum()/df.shape[0]<0.5))

163


Количество столбцов с заполнением больше 50 % всего 8 из 171. Остальные столбцы с большим количеством отсутствующих данных. Выбросил строки с отсутствующими данными. 

# EDA

In [9]:
target_var='target'

In [10]:
numeric_types = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(df.select_dtypes(include=numeric_types).columns)
object_vars = list(df.select_dtypes(include=["object"]).columns)

In [11]:
data_num = df[numerical_vars].copy()
data_obj = df[object_vars].copy()

In [12]:
data_num.head()

,target,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,-1,76698,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,-1,33058,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,-1,41040,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,-1,12,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,-1,60874,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


Все данные числовые. Текстовых данных нет

In [13]:
#Импорт метрик
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [14]:
#Отчет о точности
def quality_report(prediction, actual, verbose=True):
    report_str = "Accuracy: \t {:.3f}\n" +\
                 "Precision: \t {:.3f}\n" +\
                 "Recall: \t {:.3f}\n" +\
                 "f1_score: \t {:.3f}\n" +\
                 "ROC-AUC: \t {:.3f}"
    
    acc = accuracy_score(prediction, actual)
    precision = precision_score(prediction, actual)
    recall = recall_score(prediction, actual)
    f1 = f1_score(prediction, actual)
    rocauc = roc_auc_score(prediction, actual)
    
    quality_list = [acc, precision, recall, f1, rocauc]
    
    if verbose:
        print("\n=== Quality Report ===")
        print(report_str.format(
            accuracy_score(prediction, actual),
            precision_score(prediction, actual),
            recall_score(prediction, actual),
            f1_score(prediction, actual),
            roc_auc_score(prediction, actual)
        ))
        print("======================\n")
    
    return quality_list

In [15]:
#Разделение данных на тестовую и обучающие выборки
def data_split_train_test(data, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=test_size,
    stratify=data['target'],
    random_state=0)

    print("Train_shape:", X_train.shape, "Test shape:", X_test.shape)
    
    return X_train, X_test, y_train, y_test

In [16]:
#Логистическая регрессия

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [17]:
X_train, X_test, y_train, y_test = data_split_train_test(df)

Train_shape: (42000, 170) Test shape: (18000, 170)


In [18]:
scaler = StandardScaler()

In [19]:
data_num_wo_nan = df.dropna()

In [20]:
X_train, X_test, y_train, y_test = data_split_train_test(data_num_wo_nan)

Train_shape: (413, 170) Test shape: (178, 170)


In [21]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [22]:
df_quality= pd.DataFrame(columns=['Accuracy','Precision','Recall', 'f1_score', 'ROC-AUC'])

In [23]:
log_reg = LogisticRegression();

log_reg.fit(X_train_scaled, y_train);
print("Train quality:")
quality_report(log_reg.predict(X_train_scaled), y_train);
print("\n Test quality:")
quality_report(log_reg.predict(X_test_scaled), y_test);

df_quality.loc["log_reg", :] = quality_report(log_reg.predict(X_test_scaled), y_test);


Train quality:

=== Quality Report ===
Accuracy: 	 1.000
Precision: 	 1.000
Recall: 	 1.000
f1_score: 	 1.000
ROC-AUC: 	 1.000


 Test quality:

=== Quality Report ===
Accuracy: 	 0.944
Precision: 	 0.913
Recall: 	 0.724
f1_score: 	 0.808
ROC-AUC: 	 0.855


=== Quality Report ===
Accuracy: 	 0.944
Precision: 	 0.913
Recall: 	 0.724
f1_score: 	 0.808
ROC-AUC: 	 0.855



In [24]:
#knn
from sklearn.neighbors import KNeighborsClassifier

In [25]:
#Подбираем число соседей
#%%time
from sklearn.model_selection import GridSearchCV

param_grid = {
        'n_neighbors': range(1, 20),
        'p': range(1, 4)
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train_scaled, y_train)

print("Best CV score: {:.3f}, best CV k: {}".format(
    grid_search.best_score_, grid_search.best_estimator_.n_neighbors)
) 


test_predictions = grid_search.best_estimator_.predict(X_test_scaled)
print("Resulting test score: {:.3f}".format(f1_score(test_predictions, y_test)))

Best CV score: 0.675, best CV k: 1
Resulting test score: 0.651


In [26]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)


quality_report(knn.predict(X_train_scaled), y_train);
print("\n Test quality:")
df_quality.loc["knn", :] = quality_report(knn.predict(X_test_scaled), y_test);


=== Quality Report ===
Accuracy: 	 0.935
Precision: 	 0.491
Recall: 	 1.000
f1_score: 	 0.658
ROC-AUC: 	 0.965


 Test quality:

=== Quality Report ===
Accuracy: 	 0.927
Precision: 	 0.435
Recall: 	 1.000
f1_score: 	 0.606
ROC-AUC: 	 0.961



In [27]:
from sklearn import svm
#from sklearn.svm import SVC
lin_svm = svm.SVC(kernel='linear', C=1)
lin_svm.fit(X_train_scaled, y_train)

quality_report(lin_svm.predict(X_train_scaled), y_train);
print("\n Test quality:")
df_quality.loc["svm_1", :] = quality_report(lin_svm.predict(X_test_scaled), y_test);


=== Quality Report ===
Accuracy: 	 1.000
Precision: 	 1.000
Recall: 	 1.000
f1_score: 	 1.000
ROC-AUC: 	 1.000


 Test quality:

=== Quality Report ===
Accuracy: 	 0.938
Precision: 	 0.913
Recall: 	 0.700
f1_score: 	 0.792
ROC-AUC: 	 0.843



In [29]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
  
# определение диапазона параметров
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3);
  
# подгонка модели для поиска по сетке (ядро rbf)
grid.fit(X_train_scaled, y_train);



Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....... C=0.1, gamma=1, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....... C=0.1, gamma=1, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....... C=0.1, gamma=1, kernel=rbf, score=0.875912, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ..... C=0.1, gamma=0.1, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ..... C=0.1, gamma=0.1, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ..... C=0.1, gamma=0.1, kernel=rbf, score=0.875912, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .... C=0.1, gamma=0.01, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .... C=0.1, gamma=0.01, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=0.01, kernel=rbf ...................................
[CV] .... C=0.1, gamma=0.01, kernel=rbf, score=0.875912, total=   0.0s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ... C=0.1, gamma=0.001, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ... C=0.1, gamma=0.001, kernel=rbf, score=0.869565, total=   0.0s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] .

[CV] ... C=1000, gamma=0.01, kernel=rbf, score=0.912409, total=   0.0s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] .. C=1000, gamma=0.001, kernel=rbf, score=0.963768, total=   0.0s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] .. C=1000, gamma=0.001, kernel=rbf, score=0.963768, total=   0.0s
[CV] C=1000, gamma=0.001, kernel=rbf .................................
[CV] .. C=1000, gamma=0.001, kernel=rbf, score=0.934307, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] . C=1000, gamma=0.0001, kernel=rbf, score=0.978261, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] . C=1000, gamma=0.0001, kernel=rbf, score=0.963768, total=   0.0s
[CV] C=1000, gamma=0.0001, kernel=rbf ................................
[CV] . C=1000, gamma=0.0001, kernel=rbf, score=0.912409, total=   0.0s


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    1.7s finished


In [30]:
print(grid.best_params_) 

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}


In [31]:
from sklearn import svm
#from sklearn.svm import SVC
lin_svm_rbf = svm.SVC(kernel='rbf', C=10, gamma=0.001)
lin_svm_rbf.fit(X_train_scaled, y_train)

quality_report(lin_svm_rbf.predict(X_train_scaled), y_train);
print("\n Test quality:")
df_quality.loc["svm_best", :] =quality_report(lin_svm_rbf.predict(X_test_scaled), y_test);


=== Quality Report ===
Accuracy: 	 0.981
Precision: 	 0.868
Recall: 	 0.979
f1_score: 	 0.920
ROC-AUC: 	 0.980


 Test quality:

=== Quality Report ===
Accuracy: 	 0.949
Precision: 	 0.739
Recall: 	 0.850
f1_score: 	 0.791
ROC-AUC: 	 0.906



In [32]:
from sklearn import svm
#from sklearn.svm import SVC
lin_svm_poly = svm.SVC(kernel='poly', C=1)
lin_svm_poly.fit(X_train_scaled, y_train)

quality_report(lin_svm_poly.predict(X_train_scaled), y_train);
print("\n Test quality:")
quality_report(lin_svm_poly.predict(X_test_scaled), y_test);


=== Quality Report ===
Accuracy: 	 0.956
Precision: 	 0.660
Recall: 	 1.000
f1_score: 	 0.795
ROC-AUC: 	 0.976


 Test quality:

=== Quality Report ===
Accuracy: 	 0.916
Precision: 	 0.565
Recall: 	 0.722
f1_score: 	 0.634
ROC-AUC: 	 0.830



In [33]:
df_quality

,Accuracy,Precision,Recall,f1_score,ROC-AUC
log_reg,0.94382,0.913043,0.724138,0.807692,0.855358
knn,0.926966,0.434783,1,0.606061,0.96131
svm_1,0.938202,0.913043,0.7,0.792453,0.843243
svm_best,0.949438,0.73913,0.85,0.790698,0.906013


---